In [1]:
import numpy as np 
import pandas as pd 


In [ ]:
rows = [
    {"x": 1.2, "y": 3.4, "t": 0},
    {"x": 1.4, "y": 3.6, "t": 1},
    {"x": 1.6, "y": 3.8, "t": 2},
    {"x": 1.8, "y": 4.0, "t": 3},
    {"x": 2.6, "y": 3.8, "t": 4},
    {"x": 2.8, "y": 13.0, "t": 5},
]

for i in range(len(rows)):
    rows=
    df=pd.DataFrame(rows)
    print(df)

     x     y  t
0  2.8  13.0  5
Empty DataFrame
Columns: []
Index: []
Empty DataFrame
Columns: []
Index: []
Empty DataFrame
Columns: []
Index: []
Empty DataFrame
Columns: []
Index: []
